In [ ]:
!pip install -q transformers datasets scikit-learn pandas

In [ ]:
from google.colab import files
import os

In [ ]:
import pandas as pd
import torch
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
file_path = r"/content/Sentences_66Agree.txt"

# Use cp1252 encoding instead of utf-8
with open(file_path, 'r', encoding='cp1252') as f:
    lines = f.readlines()

# Inspect
print(lines[:5])

sentences = []
labels = []

for line in lines:
    line = line.strip()
    if not line:
        continue
    if "@positive" in line:
        label = "positive"
        text = line.replace("@positive", "").strip()
    elif "@negative" in line:
        label = "negative"
        text = line.replace("@negative", "").strip()
    elif "@neutral" in line:
        label = "neutral"
        text = line.replace("@neutral", "").strip()
    else:
        continue  # skip malformed lines

    sentences.append(text)
    labels.append(label)

print(f"Loaded {len(sentences)} sentences.")

# 3️⃣ Map labels to integers
df = pd.DataFrame({"text": sentences, "label": labels})
df['label'] = df['label'].map({"negative": 0, "neutral": 1, "positive": 2})

# Inspect
print(df.head())
print(df['label'].value_counts())


# Split into train/test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])


# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenization
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")

def preprocess(batch):
    return tokenizer(batch['text'], truncation=True, padding='max_length', max_length=128)

train_dataset = train_dataset.map(preprocess, batched=True)
test_dataset = test_dataset.map(preprocess, batched=True)

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


In [ ]:
# Load FinBERT pre-trained for sentiment classification
model = AutoModelForSequenceClassification.from_pretrained(
    "yiyanghkust/finbert-tone",
    num_labels=3,  # positive, neutral, negative
    ignore_mismatched_sizes=True
)

print(f"✓ Model loaded with {model.num_parameters():,} parameters")

In [ ]:
# Metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    f1_per_class = f1_score(labels, predictions, average=None)

    return {
        "accuracy": acc,
        "f1": f1,
        "f1_negative": f1_per_class[0],
        "f1_neutral": f1_per_class[1],
        "f1_positive": f1_per_class[2],
    }

In [ ]:
training_args = TrainingArguments(
    output_dir="./finbert_full_ft",
    num_train_epochs=2,                 # Reduce epochs from 3 → 2
    per_device_train_batch_size=32,     # Increase batch size from 16 → 32
    per_device_eval_batch_size=32,      # Match eval batch size
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    eval_strategy="steps",        # Evaluate every few steps instead of full epoch
    eval_steps=100,                     # Evaluate every 100 steps
    save_strategy="steps",              # Save checkpoints every few steps
    save_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=True,                          # Use mixed precision if GPU supports it
    gradient_accumulation_steps=1,      # Can increase if GPU memory is limited
    report_to="none",
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
print("\n🚀 Starting full fine-tuning...")
eval_results_full = trainer.evaluate()

print("-" * 40)
for key, value in eval_results_full.items():
    if key.startswith('eval_'):
        metric_name = key.replace('eval_', '').upper()
        print(f"{metric_name}: {value:.4f}")

predictions_full = trainer.predict(test_dataset)
y_pred_full = np.argmax(predictions_full.predictions, axis=-1)
y_true = predictions_full.label_ids

# Save model
print("\n💾 Saving full fine-tuned model...")
trainer.save_model("./news_classifier_full_finetuning")
tokenizer.save_pretrained("./news_classifier_full_finetuning")

In [ ]:
# Classification report
print("\n📊 Classification Report:")
print(classification_report(y_true, y_pred_full, target_names=["negative", "neutral", "positive"]))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred_full)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["negative", "neutral", "positive"])

# Plot heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["negative", "neutral", "positive"], yticklabels=["negative", "neutral", "positive"])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix - Full Fine-tuning")
plt.show()


In [ ]:
metrics_df.head()
metrics_df.columns

In [ ]:
# Separate training and evaluation logs
train_loss_df = metrics_df[metrics_df["loss"].notnull()]
eval_loss_df = metrics_df[metrics_df["eval_loss"].notnull()]

plt.figure(figsize=(8, 5))
plt.plot(train_loss_df["epoch"], train_loss_df["loss"], 'o-', label="Training Loss")
plt.plot(eval_loss_df["epoch"], eval_loss_df["eval_loss"], 's-', label="Validation Loss")

plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training vs Validation Loss")
plt.legend()
plt.grid(True)
plt.show()
